In [1]:
import collections
import itertools

import numpy as np
import pandas as pd
import scipy.sparse
import tqdm
import xswap

import analysis

In [8]:
df = pd.read_table('edges_df.tsv.gz')

np.random.seed(0)
edges_df = (
    df
    .assign(
        train=df['string'].apply(lambda x: x == 1 and np.random.rand() < 0.7).astype(int)
    )
    .filter(items=['uniprot_a', 'uniprot_b', 'train', 'string', 'ht_2014'])
)

print("{} unique genes".format(len(set(edges_df.iloc[:, 0:2].values.flatten()))))

edges_df.head()

1389 unique genes


,uniprot_a,uniprot_b,train,string,ht_2014
0,A4D1E9,A4D1E9,0,0.0,0.0
1,A4D1E9,O00144,0,0.0,0.0
2,A4D1E9,O00148,1,1.0,0.0
3,A4D1E9,O00151,0,1.0,0.0
4,A4D1E9,O00160,0,0.0,0.0


In [4]:
# Extract edge tuples
string_edges_df = edges_df.query('train == 1')
edges = zip(string_edges_df['uniprot_a'], string_edges_df['uniprot_b'])
edges = list(set(map(tuple, map(sorted, edges))))
mapped_edges, mapping, _ = (
    xswap.preprocessing.map_str_edges(edges, bipartite=False))   

# Create adjacency matrix
sp_mat = analysis.edges_to_matrix(mapped_edges)

# Create source, target degree matrices
degree = np.repeat(sp_mat.sum(axis=1), sp_mat.shape[1], axis=1) \
       + np.repeat(sp_mat.sum(axis=0), sp_mat.shape[0], axis=0)

In [5]:
# Compute features on unpermuted network
feature_mats = {
    'prior_empirical': np.zeros(sp_mat.shape),
    
    'rwr': analysis.invertible_rwr(sp_mat, 0.25),
    'mean_rwr': np.zeros(sp_mat.shape),
    'p_rwr': np.zeros(sp_mat.shape),
    
    'jaccard': (sp_mat@sp_mat) / (degree - sp_mat@sp_mat),
    'mean_jaccard': np.zeros(sp_mat.shape),
    'p_jaccard': np.zeros(sp_mat.shape),
}
# Compute RWR p-value
n_perms = 1000
perm_edges = mapped_edges.copy()
for i in tqdm.tnrange(n_perms):
    # Permute edges
    perm_edges, _ = xswap.permute_edge_list(perm_edges, allow_self_loops=True, 
                                            allow_antiparallel=False, seed=i)
    perm_mat = analysis.edges_to_matrix(perm_edges)
    feature_mats['prior_empirical'] += perm_mat
    
    # Compute RWR on permuted network
    perm_rwr = analysis.invertible_rwr(perm_mat.toarray(), 0.25)
    feature_mats['mean_rwr'] += perm_rwr
    feature_mats['p_rwr'] += (perm_rwr >= feature_mats['rwr'])
    
    # Compute Jaccard similarity on permuted network
    A2 = perm_mat@perm_mat
    perm_jac = A2 / (degree - A2)
    feature_mats['mean_jaccard'] += perm_jac
    feature_mats['p_jaccard'] += (perm_jac >= feature_mats['jaccard'])

# Normalize features to number of permutations
for feature in ['mean_rwr', 'p_rwr', 'mean_jaccard', 'p_jaccard', 'prior_empirical']:
    data = feature_mats[feature] / n_perms
    if scipy.sparse.issparse(data):
        data = data.toarray().flatten()
    else:
        data = np.array(data).flatten()
    feature_mats[feature] = data
    
feature_dict = {k: np.array(v).flatten() for k, v in feature_mats.items()}

# Unmap RWR and add to DataFrame
max_id = max(mapping.values())
reversed_map = {v: k for k, v in mapping.items()}
feature_dict['mapped_source'], feature_dict['mapped_target'] = zip(*itertools.product(
    range(max_id+1), range(max_id+1)))

KeyboardInterrupt: 

In [ ]:
# Unmap RWR and add to DataFrame
max_id = max(mapping.values())
reversed_map = {v: k for k, v in mapping.items()}
mapped_source, mapped_target = zip(*itertools.product(range(max_id+1), range(max_id+1)))
mapped_rwr_df = (
    pd.DataFrame
    .from_dict(feature_dict)
    .assign(
        uniprot_a=lambda df: df['mapped_source'].map(reversed_map),
        uniprot_b=lambda df: df['mapped_target'].map(reversed_map)
    )
    .filter(items=['uniprot_a', 'uniprot_b', 'prior_empirical', 
                   'rwr', 'mean_rwr', 'p_rwr', 
                   'jaccard', 'mean_jaccard', 'p_jaccard'])
)
# Dictionary of node to degree
uniprot_to_degree = collections.Counter(edges_df.groupby('uniprot_a')['train'].sum().to_dict())

df = (
    edges_df
    .merge(mapped_rwr_df, on=['uniprot_a', 'uniprot_b'], how='left')
    .fillna(0)
    .assign(
        source_degree=lambda df: df['uniprot_a'].map(uniprot_to_degree),
        target_degree=lambda df: df['uniprot_b'].map(uniprot_to_degree),
        mean_degree=lambda df: np.sqrt(df['source_degree'] * df['target_degree']),
    )
    .filter(['uniprot_a', 'uniprot_b', 'source_degree', 'target_degree', 'mean_degree', 'train', 
             'string', 'ht_2014', 'prior_empirical', 'rwr', 'mean_rwr', 'p_rwr', 'jaccard', 
             'mean_jaccard', 'p_jaccard',])
)

df.to_csv('p_vs_rank.tsv.xz', compression='xz', sep='\t', index=False)
df.head()